In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [21]:
inventory_data = pd.read_excel('./data/inventory_data_new.xlsx')

In [22]:
inventory_data.head()

,material,unitats_2022,vendes_2022,preu_venda_unitari_2022,unitats_2023,vendes_2023,preu_venda_unitari_2023,variacio_preu_venda_unitari_2022_2023,proporcio_variacio_preu_venda_unitari_2022_2023,data_darrera_entrada,dies_ultima_entrada,data_darrera_sortida,dies_ultima_sortida,diferencia_entrada_sortida,stock_final_2023,valor_total_stock_2023,cost_unitari_stock_2023
0,100500401.0,350.0,766.77,2.190771,300.0,931.00,3.103333,0.912562,0.416548,1970-01-01 01:00:45.208,83.0,1970-01-01 01:00:45.219,72.0,11.0,NaN,NaN,NaN
1,100500601.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1970-01-01 01:00:44.551,740.0,1970-01-01 01:00:44.551,740.0,0.0,NaN,NaN,NaN
2,114230204.0,140000.0,8072.60,0.057661,70000.0,4489.80,0.064140,0.006479,0.112355,1970-01-01 01:00:45.258,33.0,1970-01-01 01:00:45.272,19.0,14.0,NaN,NaN,NaN
3,114230821.0,40000.0,1945.60,0.048640,NaN,NaN,NaN,NaN,NaN,1970-01-01 01:00:44.581,710.0,1970-01-01 01:00:44.588,703.0,7.0,NaN,NaN,NaN
4,114230822.0,1018500.0,54892.49,0.053895,885500.0,58951.87,0.066575,0.012679,0.235256,1970-01-01 01:00:45.351,-60.0,1970-01-01 01:00:45.336,-45.0,-15.0,66500.0,2292.96,0.034481


In [23]:
# Passar a data les columnes que ho requereixen
inventory_data['data_darrera_entrada'] = pd.to_datetime(inventory_data['data_darrera_entrada'], format='%Y-%m-%d')
inventory_data['data_darrera_sortida'] = pd.to_datetime(inventory_data['data_darrera_sortida'], format='%Y-%m-%d')

In [24]:
categorical_cols = ['material']
date_cols = ['data_darrera_entrada', 'data_darrera_sortida']
numerical_cols = list(set(inventory_data.columns) - set(categorical_cols) - set(date_cols))
for col in numerical_cols:
	inventory_data[col] = pd.to_numeric(inventory_data[col])

In [25]:
inventory_data.columns

Index(['material', 'unitats_2022', 'vendes_2022', 'preu_venda_unitari_2022',
       'unitats_2023', 'vendes_2023', 'preu_venda_unitari_2023',
       'variacio_preu_venda_unitari_2022_2023',
       'proporcio_variacio_preu_venda_unitari_2022_2023',
       'data_darrera_entrada', 'dies_ultima_entrada', 'data_darrera_sortida',
       'dies_ultima_sortida', 'diferencia_entrada_sortida', 'stock_final_2023',
       'valor_total_stock_2023', 'cost_unitari_stock_2023'],
      dtype='object')

In [26]:
# Fill missing values and NAN with 0

inventory_data = inventory_data.fillna(0)

In [29]:
def calcular_indice_deterioro(data):
    # Normalizar las diferencias de unidades vendidas
    data['delta_unitats'] = (data['unitats_2023'] - data['unitats_2022']) / (data['unitats_2022'] + 1)
    
    # Normalizar la variación del precio unitario
    data['delta_preu_venda_unitari'] = data['variacio_preu_venda_unitari_2022_2023'] / (data['preu_venda_unitari_2022'] + 0.01)
    
    # Índice de rotación del inventario
    data['rotacion_inventario'] = data['unitats_2023'] / (data['stock_final_2023'] + 1)
    
    # Días sin actividad (venta o entrada)
    data['inactividad'] = data['dies_ultima_sortida'] + data['dies_ultima_entrada']
    
    # Índice de deterioro basado en una combinación de factores
    data['indice_deterioro'] = (
        -0.3 * data['delta_unitats'] +             # Impacto negativo si hay reducción en ventas
        -0.2 * data['delta_preu_venda_unitari'] +  # Impacto negativo si hay reducción en el precio de venta
         0.2 * np.log1p(data['inactividad']) +    # Mayor inactividad, mayor deterioro
        -0.3 * data['rotacion_inventario']         # Menor rotación indica posible deterioro
    )
    
    return data

inventory_data = calcular_indice_deterioro(inventory_data)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: divide by zero encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [30]:
inventory_data.head(100)

,material,unitats_2022,vendes_2022,preu_venda_unitari_2022,unitats_2023,vendes_2023,preu_venda_unitari_2023,variacio_preu_venda_unitari_2022_2023,proporcio_variacio_preu_venda_unitari_2022_2023,data_darrera_entrada,...,dies_ultima_sortida,diferencia_entrada_sortida,stock_final_2023,valor_total_stock_2023,cost_unitari_stock_2023,delta_unitats,delta_preu_venda_unitari,rotacion_inventario,inactividad,indice_deterioro
0,100500401.0,350.0,766.77,2.190771,300.0,931.00,3.103333,0.912562,0.416548,1970-01-01 01:00:45.208000,...,72.0,11.0,0.0,0.00,0.000000,-0.142450,0.414655,300.000000,155.0,-89.030225
1,100500601.0,0.0,0.00,0.000000,0.0,0.00,0.000000,0.000000,0.000000,1970-01-01 01:00:44.551000,...,740.0,0.0,0.0,0.00,0.000000,0.000000,0.000000,0.000000,1480.0,1.460095
2,114230204.0,140000.0,8072.60,0.057661,70000.0,4489.80,0.064140,0.006479,0.112355,1970-01-01 01:00:45.258000,...,19.0,14.0,0.0,0.00,0.000000,-0.499996,0.095750,70000.000000,52.0,-20999.075093
3,114230821.0,40000.0,1945.60,0.048640,0.0,0.00,0.000000,0.000000,0.000000,1970-01-01 01:00:44.581000,...,703.0,7.0,0.0,0.00,0.000000,-0.999975,0.000000,0.000000,1413.0,1.750828
4,114230822.0,1018500.0,54892.49,0.053895,885500.0,58951.87,0.066575,0.012679,0.235256,1970-01-01 01:00:45.351000,...,-45.0,-15.0,66500.0,2292.96,0.034481,-0.130584,0.198437,13.315589,-105.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,133066463.0,0.0,0.00,0.000000,0.0,0.00,0.000000,0.000000,0.000000,1970-01-01 01:00:44.245000,...,1007.0,39.0,0.0,0.00,0.000000,0.000000,0.000000,0.000000,2053.0,1.525509
96,133066482.0,2880000.0,92897.28,0.032256,72000.0,2640.96,0.036680,0.004424,0.137153,1970-01-01 01:00:44.872000,...,191.0,228.0,0.0,0.00,0.000000,-0.975000,0.104695,72000.000000,610.0,-21598.445420
97,133066488.0,0.0,0.00,0.000000,0.0,0.00,0.000000,0.000000,0.000000,1970-01-01 01:00:44.316000,...,416.0,559.0,0.0,0.00,0.000000,0.000000,0.000000,0.000000,1391.0,1.447699
98,133066491.0,2340000.0,72599.04,0.031025,133200.0,4717.19,0.035414,0.004389,0.141469,1970-01-01 01:00:44.804000,...,347.0,140.0,0.0,0.00,0.000000,-0.943077,0.106986,133200.000000,834.0,-39958.392988
